# Notebook Imports

In [4]:
import numpy as np
import pandas as pd

# Constants

In [7]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [8]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [9]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [11]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 16,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [13]:
sparse_train_data[-5:]

array([[5795, 1834,    0,    1],
       [5795, 1905,    0,    2],
       [5795, 2105,    0,    1],
       [5795, 2257,    0,    1],
       [5795, 2463,    0,    1]])

In [14]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 265577
Nr of rows in test file 110578


In [22]:
print('Nr of emails in training file', np.unique(sparse_train_data[:,0]).size)

Nr of emails in training file 4014


In [23]:
print('Nr of emails in testing file', np.unique(sparse_test_data[:,0]).size)

Nr of emails in testing file 1723


## How to create an Empty DataFrame

In [29]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [30]:
len(column_names)

2502

In [31]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [38]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True) #Fill empty dataframe with 0

In [39]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [43]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    '''
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix --numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse martrix. Default: 1st column
    word_idx --position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    '''
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]): # 0 -> nr of rows in sparse matrix
        
        # Store all the values in a particular row in four separate variables
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        # Selecting single cell and setting its values
        # Grabing data from sparse matrix and populating it in empty dataFrame
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    #
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [44]:
%%time
full_train_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 10.8 s


In [45]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
12,1,0,2,1,1,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
15,1,3,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19,1,1,2,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5783,0,2,2,2,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,0,2,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5792,0,2,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5793,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
